# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.12 AI应用开发新范式：借助AI完成AI应用开发（上）

In [573]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import base64
import email
from email import policy
from email.parser import BytesParser
from email.mime.text import MIMEText

from bs4 import BeautifulSoup
import dateutil.parser as parser

from gptLearning import *

In [62]:
SCOPES = ['https://www.googleapis.com/auth/gmail.send','https://www.googleapis.com/auth/gmail.readonly']
flow = InstalledAppFlow.from_client_secrets_file(
                'client_web1.json', SCOPES)
creds = flow.run_local_server(port=8899, access_type='offline', prompt='consent')

with open('token.json', 'w') as token:
    token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=597598790553-j4tj274ajpeta15s9lc0sffmlqmbhi9q.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8899%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=f8hxnzVwVisxQUsaFphgp7lc3KNECw&access_type=offline&prompt=consent


### 1.初阶策略：用Chat模型辅助进行代码编写

#### 1.1 借助ChatGPT协助编写外部函数

&emsp;&emsp;如何才能借助Chat模型来帮我们提高AI应用开发效率呢？一个最基础的策略，就是尝试让Chat模型完成对应功能的外部函数的编写。当然，这里我们可以整理需求后直接在ChatGPT中提问，令其生成外部函数的代码，然后复制到当前代码环境中进行测试和修改。考虑到ChatGPT本身强大的代码编写功能，以及当前并不复杂的代码需求（主要是测试功能能否实现、而不是优化代码执行流程），外加大多数API的功能本身都是包含在ChatGPT知识库中的，因此让ChatGPT编写调用相关API的函数并不会太难。例如，还是围绕Gmail API的调用，我们希望编写一个函数能够查看最近接收到的5封邮件，则可以按照如下方式对ChatGPT进行提问：

我现在已经获取了Gmail API并完成了OAuth 2.0客户端和授权，并将查看邮件的凭据保存为token.json文件。我现在想要编写一个函数来查阅最近的n封邮件，函数要求如下：         
1.函数参数为n和userId，其中userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件，而n则是整数，代表需要查询的邮件个数；        
2.函数返回结果为一个包含多个字典的列表，并用json格式进行表示，其中一个字典代表一封邮件信息，每个字典中需要包含邮件的发件人、发件时间、邮件主题和邮件内容四个方面信息；        
3.请将全部功能封装在一个函数内；       
4.请在函数编写过程中，帮我编写详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/2a4819ba4c4b196227d53c0a828cf06.png" alt="2a4819ba4c4b196227d53c0a828cf06" style="zoom:33%;" />

&emsp;&emsp;需要注意的是，这里我们提问的最终目标是希望ChatGPT帮我们直接创建能满足外部函数格式要求的函数。以下是ChatGPT返回的代码结果并尝试运行：

In [5]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import base64
import email
from bs4 import BeautifulSoup
import dateutil.parser as parser
import json

def retrieve_emails(n, user_id='me'):
    """
    获取指定数量的最近邮件。

    参数:
    n: 要检索的邮件的数量。这应该是一个整数。
    user_id: 要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'From': 发件人的邮箱地址。
    'Date': 邮件的发送日期。
    'Subject': 邮件的主题。
    'Snippet': 邮件的摘要（前100个字符）。
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')

    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)

    # 获取邮件列表
    results = service.users().messages().list(userId=user_id).execute()
    messages = results.get('messages', [])[:n]

    emails = []
    for message in messages:
        msg = service.users().messages().get(userId=user_id, id=message['id']).execute()

        # 解码邮件内容
        payload = msg['payload']
        headers = payload.get("headers")
        parts = payload.get("parts")

        data = {}

        if headers:
            for d in headers:
                name = d.get("name")
                if name.lower() == "from":
                    data['From'] = d.get("value")
                if name.lower() == "date":
                    data['Date'] = parser.parse(d.get("value")).strftime('%Y-%m-%d %H:%M:%S')
                if name.lower() == "subject":
                    data['Subject'] = d.get("value")

        if parts:
            for part in parts:
                mimeType = part.get("mimeType")
                body = part.get("body")
                data_decoded = base64.urlsafe_b64decode(body.get("data")).decode()
                if mimeType == "text/plain":
                    data['Snippet'] = data_decoded
                elif mimeType == "text/html":
                    soup = BeautifulSoup(data_decoded, "html.parser")
                    data['Snippet'] = soup.get_text()

        emails.append(data)

    # 返回邮件列表
    return json.dumps(emails, indent=2)

能够看出，ChatGPT能够完整编写实现该功能的函数，并且在函数中给出了可能出现的非常多种不同情况，该函数还是非常完整详尽的。接下来我们测试函数能否顺利执行：

In [33]:
msg = retrieve_emails(5)

In [35]:
msg

'[\n  {\n    "From": "\\"\\u7aef\\u6728\\u5929\\" <2323365771@qq.com>",\n    "Subject": "Re: \\u5173\\u4e8e\\u660e\\u5929\\u7684\\u4f1a\\u8bae",\n    "Date": "2023-07-21 22:39:17",\n    "Snippet": "\\u597d\\u7684\\uff0c\\u6536\\u5230\\u53d1\\u81ea\\u6211\\u7684iPhone------------------ \\u539f\\u59cb\\u90ae\\u4ef6 ------------------\\u53d1\\u4ef6\\u4eba: ksken166 <ksken166@gmail.com>\\u53d1\\u9001\\u65f6\\u95f4: 2023\\u5e747\\u670821\\u65e5 22:39\\u6536\\u4ef6\\u4eba: 2323365771 <2323365771@qq.com>\\u4e3b\\u9898: Re: \\u5173\\u4e8e\\u660e\\u5929\\u7684\\u4f1a\\u8bae\\u4f60\\u597d\\u9648\\u660e\\uff0c\\u8bf7\\u4f60\\u660e\\u5929\\u65e9\\u4e0a9\\u70b9\\u534a\\u6765\\u6211\\u7684\\u529e\\u516c\\u5ba4\\u5f00\\u4f1a\\uff0c\\u6211\\u4eec\\u4f1a\\u5546\\u91cf\\u4e0b\\u534a\\u5e74\\u7684\\u6280\\u672f\\u5f00\\u53d1\\u8ba1\\u5212\\u3002\\u8c22\\u8c22\\uff0c\\u4e5d\\u5929"\n  },\n  {\n    "From": "ksken166@gmail.com",\n    "Subject": "\\u5173\\u4e8e\\u660e\\u5929\\u7684\\u4f1a\\u8bae",\n    "Date

然后测试这样的函数结果能否被Chat模型解读：

In [37]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "这是我的Gmail邮箱最近五封邮件的内容：%s" % msg},
    {"role": "system", "content": "邮件内容是由Gmail API获取"},
    {"role": "user", "content": "请问我的Gmail最近五封邮件是谁发送的，都有什么内容？"}
  ]
)
response.choices[0].message['content']

'1. 发件人：端木天\n   主题：关于明天的会议\n   时间：2023-07-21 22:39:17\n   内容：明确收到并确认你发出的会议邀请。会议内容关于明天早上9点半在你的办公室进行的技术开发计划商议。\n\n2. 发件人：你自己 \n   主题：关于明天的会议\n   时间：2023-07-21 07:38:55\n   内容：未明确给出，可能是与端木天讨论的技术开发计划相关。\n\n3. 发件人：端木天\n   主题：关于明天开会的通知\n   时间：2023-07-21 22:28:34\n   内容：明确收到并确认你发出的会议通知。\n\n4. 发件人：你自己\n   主题：关于明天开会的通知\n   时间：2023-07-21 07:27:55\n   内容：未明确给出，可能是与端木天提醒的开会通知相关。\n\n5. 发件人：你自己 \n   主题：关于明天早上9点半开会的通知\n   时间：2023-07-21 07:24:13\n   内容：未明确给出，可能是关于明天早上9:30会议的详细信息或安排。'

能够发现Chat模型能够顺利获取这5封邮件的信息。这里信息看似混乱，实际上是测试Gmail API功能测试时和陈明反复发送的多封邮件。

&emsp;&emsp;接下来我们继续测试，这个由ChatGPT帮我们编写的retrieve_emails函数，其参数说明能否被auto_functions函数正常识别，并创建functions参数：

In [38]:
functions_list = [retrieve_emails]

In [40]:
functions = auto_functions(functions_list)
functions

[{'name': 'retrieve_emails',
  'description': '获取指定数量的最近邮件',
  'parameters': {'type': 'object',
   'properties': {'n': {'type': 'integer', 'description': '要检索的邮件的数量'},
    'user_id': {'type': 'string',
     'description': '要检索邮件的用户的ID',
     'default': 'me'}},
   'required': ['n']}}]

接下来测试functions函数说明能否被Chat模型正确识别：

In [41]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[{"role": "user", "content": '请帮我查下最近3封邮件的邮件内容'}],
        functions=functions,
        function_call="auto",  
    )

In [42]:
response

<OpenAIObject chat.completion id=chatcmpl-7f2HnbqwVRwGGzKtbTdT8txaQMepZ at 0x15a08c04220> JSON: {
  "id": "chatcmpl-7f2HnbqwVRwGGzKtbTdT8txaQMepZ",
  "object": "chat.completion",
  "created": 1690014591,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "retrieve_emails",
          "arguments": "{\n  \"n\": 3\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 101,
    "completion_tokens": 15,
    "total_tokens": 116
  }
}

能够发现，Chat模型能够顺利识别functions中函数说明，并在当前对话情景中正确创建必要参数n=3。接下来我们直接将这个函数带入多轮对话进行进一步测试：

In [43]:
chat_with_model(functions_list=functions_list)

模型回答: 你好！有什么我可以帮助你的吗？


您还有其他问题吗？(输入退出以结束对话):  请帮我查阅最近两封邮件的邮件内容


模型回答: 最近的两封邮件如下：

1. 来自端木天的邮件（发信地址：2323365771@qq.com），发送时间为2023年7月21日22:39。
   标题为：回复：关于明天的会议。
   内容如下：好的，收到。发自我的iPhone。

2. 来自ksken166@gmail.com的邮件，发送时间为2023年7月21日07:38。
   标题为：关于明天的会议。
   邮件内容未提供。


您还有其他问题吗？(输入退出以结束对话):  退出


通过多轮对话测试，当然这也说明由ChatGPT创建的函数，能够在当前的auto_functions+run_conversation函数支持的开发流程中顺利加入到外部函数库中。而如果外部函数能够借助ChatGPT进行创建，毫无疑问此时AI应用的开发效率也将大幅提升。在后续的课程中，我们也将尽可能尝试借助大语言模型本身帮我们创建外部函数。

&emsp;&emsp;当然，这里需要重点强调的是，在上述借助ChatGPT帮我们编写外部函数的流程中，最重要的有三点，其一是根据开发者的经验先获取了API相关凭据，跑通了整个API授权流程；其二是在非常了解Function calling功能以及我们设置的auto_functions+run_conversation开发流程基础之上，对ChatGPT进行了合理的提示；其三则是在获得了ChatGPT编写的函数之后，借助auto_functions进行外部函数功能验证。需要注意的是，只有我们拥有了这三方面的基础能力，才能够真正顺利应用ChatGPT代码能力帮我们完成外部函数编写，否则，无论是没有完成授权、还是没有正确的提示、或者没有函数的验证过程，我们都无法驾驭ChatGPT代码创建能力为我们所用。

#### 1.2 借助Chat模型实现本地代码高效编写与验证

&emsp;&emsp;虽然借助ChatGPT帮我们编写外部函数已经能够极大程度加快AI应用的开发效率，但作为“追求极致效率”的开发者，我们所希望的是能够更进一步，即能否省略每次都向ChatGPT提问然后复制粘贴代码到本地进行验证的这个环节呢？能否让创建外部函数更加自动化，即通过自然语言提示，直接在代码环境中创建外部函数代码，并自动进行测试和封装呢？

- 测试Chat模型输出结果直接转化为代码并运行

&emsp;&emsp;要做到这点，首先我们需要跑通利用Chat模型创建函数并直接运行这一流程。我们知道，Chat模型的输入和输出都是字符串，因此若希望Chat模型输出结果直接转化为可以运行的外部函数，不仅需要合理的提示，还需要补充一些可以提取字符串中python代码并直接运行的方法。

&emsp;&emsp;首先，我们先通过一个简单的示例，查看当前使用的gpt-4-0613模型是否能在合理的提示下，创建符合外部函数要求的函数：

In [187]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[{"role": "system", "content": "你是一个python代码编辑器，你的功能是输出python代码，请勿输出任何和python代码无关的内容"},
            {"role": "user", "content": "请帮我定义一个python函数，输出Hello world字符串，请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档。"}
  ]
)

In [188]:
response.choices[0].message['content']

'```python\ndef print_hello_world():\n    """\n    这是一个简单的函数，其功能是输出"Hello world"字符串。\n\n    说明：\n    本函数没有参数，也没有返回值。\n    """\n    print("Hello world")\n```\n当运行这个函数print_hello_world()时， it will print "Hello world" to the console.'

需要注意的是，此时Chat模型输出的字符串其实是代表一个markdown格式对象。这里我们可以通过修改提示让Chat模型更加专注的只生成Python代码，但实际测试结果发现Chat模型很难理解“只输出python代码”这一提示，哪怕是利用Few-shot提示方法，也无法解决该问题。所以一种更高效的解决问题的方法是直接在上述字符串中通过正则表达式提取出只包含Python代码的字符串。

&emsp;&emsp;当然，在提取之前，我们可以先简单查看上述markdown内容，我们可以通过如下代码先将上述字符串的全部内容在本地保存为一个名为helloworld的md文件，并查看文件内容：

In [190]:
s = response.choices[0].message['content']

In [199]:
s

'```python\ndef print_hello_world():\n    """\n    这是一个简单的函数，其功能是输出"Hello world"字符串。\n\n    说明：\n    本函数没有参数，也没有返回值。\n    """\n    print("Hello world")\n```\n当运行这个函数print_hello_world()时， it will print "Hello world" to the console.'

In [191]:
with open('helloworld.md', 'a', encoding='utf-8') as f:
    f.write(s)

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/e2b0b9cf2d91c61f074fff80fd36860.png" alt="e2b0b9cf2d91c61f074fff80fd36860" style="zoom:33%;" />

能够看出，Chat模型创建的函数本身并没有任何问题。接下来我们考虑在字符串s中单独提取python部分代码，提取函数如下：

In [273]:
def extract_code(s):
    """
    如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
    否则，返回原字符串。

    参数:
    s: 输入的字符串。

    返回:
    提取出的代码部分，或原字符串。
    """
    # 判断字符串是否是Markdown格式
    if '```python' in s or 'Python' in s or'PYTHON' in s:
        # 找到代码块的开始和结束位置
        code_start = s.find('def')
        code_end = s.find('```\n', code_start)
        # 提取代码部分
        code = s[code_start:code_end]
    else:
        # 如果字符串不是Markdown格式，返回原字符串
        code = s

    return code

测试在s上的执行结果：

In [223]:
code_s = extract_code(s)
code_s

'def print_hello_world():\n    """\n    这是一个简单的函数，其功能是输出"Hello world"字符串。\n\n    说明：\n    本函数没有参数，也没有返回值。\n    """\n    print("Hello world")\n'

能够发现，此时函数就能完整的提取出s中包含的代码部分，并将其保存为一个字符串。这里需要注意，该函数包含一个判别字符串是否是markdown格式的过程，是因为有时Chat模型也会创建一个只包含代码的字符串，而此时该字符串形式类似于code_s，并没有\```python等字样，此时函数将返回原始结果——即一个包含python函数的字符串：

In [203]:
extract_code(code_s)

'def print_hello_world():\n    """\n    这是一个简单的函数，其功能是输出"Hello world"字符串。\n\n    说明：\n    本函数没有参数，也没有返回值。\n    """\n    print("Hello world")\n'

而对于一个用字符串表示的python程序，我们可以通过如下方式将其写入本地py文件并进行代码查看：

In [224]:
code_s

'def print_hello_world():\n    """\n    这是一个简单的函数，其功能是输出"Hello world"字符串。\n\n    说明：\n    本函数没有参数，也没有返回值。\n    """\n    print("Hello world")\n'

In [225]:
with open('helloworld.py', 'w', encoding='utf-8') as f:
    f.write(code_s)

此时就会在本地创建一个保存了print_hello_world函数的py文件：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/fa78bc8a89b623228868ed6d9efa6dc.png" alt="fa78bc8a89b623228868ed6d9efa6dc" style="zoom:33%;" />

该文件相当于是函数的备份，我们可以直接通过如下形式查看这个py文件的内容：

In [228]:
with open('helloworld.py', encoding='utf-8') as f:
    content = f.read()
print(content)

def print_hello_world():
    """
    这是一个简单的函数，其功能是输出"Hello world"字符串。

    说明：
    本函数没有参数，也没有返回值。
    """
    print("Hello world")



当然保存一份该函数的py文件主要是用于函数备份，方便跨操作空间对其进行调用（直接import即可）。而在本地，我们可以直接exec函数执行这个包装在字符串中的函数代码：

In [229]:
exec(code_s)

执行后，本地即可查看code_s代码定义的函数了：

In [230]:
print_hello_world

<function __main__.print_hello_world()>

也可以直接执行该函数：

In [232]:
print_hello_world()

Hello world


接下来我们定义一个功能更加完整的extract_function_code函数。该函数可以同时在字符串中提取python代码，并提取该段代码的函数名称，同时对该函数进行同名py文件的本地保存，并在当前环境下执行该代码（定义该函数），同时可以选择打印函数的全部信息或者只是打印函数的名称：

In [6]:
def extract_function_code(s, detail=0):
    """
    函数提取函数，同时执行函数内容，可以选择打印函数信息
    """
    def extract_code(s):
        """
        如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
        否则，返回原字符串。

        参数:
        s: 输入的字符串。

        返回:
        提取出的代码部分，或原字符串。
        """
        # 判断字符串是否是Markdown格式
        if '```python' in s or 'Python' in s or'PYTHON' in s:
            # 找到代码块的开始和结束位置
            code_start = s.find('def')
            code_end = s.find('```\n', code_start)
            # 提取代码部分
            code = s[code_start:code_end]
        else:
            # 如果字符串不是Markdown格式，返回原字符串
            code = s

        return code
    
    # 提取代码字符串
    code = extract_code(s)
    
    # 提取函数名称
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    # 将函数写入本地
    with open('%s.py' % function_name, 'w', encoding='utf-8') as f:
        f.write(code)
    
    # 执行该函数
    try:
        exec(code, globals())
    except Exception as e:
        print("An error occurred while executing the code:")
        print(e)
    
    # 打印函数名称
    if detail == 0:
        print("The function name is:%s" % function_name)
    
    if detail == 1:
        with open('%s.py' % function_name, encoding='utf-8') as f:
            content = f.read()
        print(content)

函数测试效果如下：

In [38]:
s

'```python\ndef print_hello_world():\n    """\n    这是一个简单的函数，其功能是输出"Hello world"字符串。\n\n    说明：\n    本函数没有参数，也没有返回值。\n    """\n    print("Hello world")\n```\n当运行这个函数print_hello_world()时， it will print "Hello world" to the console.'

In [39]:
extract_function_code(s)

The function name is:print_hello_world


In [40]:
print_hello_world?

Signature: print_hello_world()
Docstring:
这是一个简单的函数，其功能是输出"Hello world"字符串。

说明：
本函数没有参数，也没有返回值。
File:      Dynamically generated function. No source code available.
Type:      function

In [41]:
extract_function_code(s, detail=1)

def print_hello_world():
    """
    这是一个简单的函数，其功能是输出"Hello world"字符串。

    说明：
    本函数没有参数，也没有返回值。
    """
    print("Hello world")



有了该函数，后续我们即可更加便捷的将Chat模型输出结果一键进行函数提取、保存和运行。

- 借助Chat函数编写统计邮箱全部邮件个数的函数

&emsp;&emsp;在这个流程基础之上，接下来我们尝试引导让Chat函数直接编写符合要求的外部函数——一个用于统计邮箱全部邮件个数的函数。这里考虑到代码创建的稳健性，我们采用Few-shot的方式对其进行提示，即以get_latest_email为例，教会模型如何创建一个类似的用于统计全部邮件个数的函数。此前定义的get_latest_email函数如下：

In [7]:
def get_latest_email(userId):
    """
    查询Gmail邮箱中最后一封邮件信息
    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\
    注意，当查询我的邮箱时，userId需要输入'me'；
    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')
    
    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)
    
    # 列出用户的一封最新邮件
    results = service.users().messages().list(userId=userId, maxResults=1).execute()
    messages = results.get('messages', [])

    # 遍历邮件
    for message in messages:
        # 获取邮件的详细信息
        msg = service.users().messages().get(userId='me', id=message['id']).execute()
        
    return json.dumps(msg)

为了更加灵活的进行函数源码和字符串的转化，我们可以通过inspect.getsource方式直接提取上述函数的代码并采用字符串格式进行输出：

In [90]:
code = inspect.getsource(get_latest_email)

In [96]:
code

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

当然，也可以考虑将其写入本地：

In [91]:
# 写入本地
with open('%s.py' % 'get_latest_email', 'w', encoding='utf-8') as f:
    f.write(code)

从而方便下次调用其源码字符串：

In [93]:
with open('%s.py' % 'get_latest_email', encoding='utf-8') as f:
    content = f.read()

In [94]:
content

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

接下来是具体的Few-shot提示过程：

In [98]:
system_content = "我现在已完成Gmail API授权，授权文件为本地文件token.json。"

In [99]:
user_example_content = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：\
                        1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                        2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                        3.请将全部功能封装在一个函数内；\
                        4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [100]:
with open('%s.py' % 'get_latest_email', encoding='utf-8') as f:
    assistant_example_content = f.read()

In [101]:
assistant_example_content

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

In [80]:
user_content = "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：\
                1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                2.函数返回结果是当前邮件总数，返回结果本身必须是一个json格式对象；\
                3.请将全部功能封装在一个函数内；\
                4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [102]:
messages=[{"role": "system", "content": system_content},
          {"role": "user", "name":"example_user", "content": user_example_content},
          {"role": "assistant", "name":"example_assistant", "content": assistant_example_content},
          {"role": "user", "name":"example_user", "content": user_content}]

这里需要注意，Chat模型的提示方法和ChatGPT的提示方法有很大的区别，哪怕是为了传递相同的意思，二者有效的提示方法可能也会有很大的区别，因此具体如何提示需要勤加练习并反复多次进行测试：

In [103]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=messages
)

In [104]:
s = response.choices[0].message['content']
s

'def get_email_counts(userId=\'me\'):\n    """\n    查询Gmail邮箱中邮件总数\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：邮件总数，返回结果本身必须是一个json格式对象\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n\n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 列出用户的所有邮件\n    results = service.users().messages().list(userId=userId).execute()\n    messages = results.get(\'messages\', [])\n\n    # 获得邮件数量\n    email_counts = len(messages)\n\n    return json.dumps({"email_counts": email_counts})\n'

In [106]:
extract_function_code(s, detail=0)

The function name is:get_email_counts


In [107]:
extract_function_code(s, detail=1)

def get_email_counts(userId='me'):
    """
    查询Gmail邮箱中邮件总数
    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\
    注意，当查询我的邮箱时，userId需要输入'me'；
    :return：邮件总数，返回结果本身必须是一个json格式对象
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')

    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)

    # 列出用户的所有邮件
    results = service.users().messages().list(userId=userId).execute()
    messages = results.get('messages', [])

    # 获得邮件数量
    email_counts = len(messages)

    return json.dumps({"email_counts": email_counts})



此时就完成了函数的提取和定义，接下来测试该函数能否顺利运行：

In [110]:
get_email_counts?

Signature: get_email_counts(userId='me')
Docstring:
查询Gmail邮箱中邮件总数
:param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，    注意，当查询我的邮箱时，userId需要输入'me'；
:return：邮件总数，返回结果本身必须是一个json格式对象
File:      Dynamically generated function. No source code available.
Type:      function

In [112]:
get_email_counts()

'{"email_counts": 14}'

能够发现，该函数能够正确统计当前邮箱的邮件个数。接下来进入函数是否可以作为外部函数的测试环节。首先第一步是测试该函数能否能被顺利的转化为functions参数：

In [113]:
functions_list = [get_email_counts]

In [114]:
functions = auto_functions(functions_list)
functions

[{'name': 'get_email_counts',
  'description': '查询Gmail邮箱中邮件总数',
  'parameters': {'type': 'object',
   'properties': {'userId': {'type': 'string',
     'description': "需要查询的邮箱ID，查询我的邮箱时，需要输入'me'"}},
   'required': ['userId']}}]

第二步是测试functions函数说明能否被Chat模型正确识别：

In [115]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[{"role": "user", "content": '请帮我查下邮箱里现在总共有多少封邮件'}],
        functions=functions,
        function_call="auto",  
    )

In [116]:
response

<OpenAIObject chat.completion id=chatcmpl-7f6pcBEMRBkTKXUfUfNVbGuZcsNPp at 0x2632e076cf0> JSON: {
  "id": "chatcmpl-7f6pcBEMRBkTKXUfUfNVbGuZcsNPp",
  "object": "chat.completion",
  "created": 1690032064,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_email_counts",
          "arguments": "{\n  \"userId\": \"me\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 16,
    "total_tokens": 105
  }
}

最后来测试下带入多轮对话函数的对话效果：

In [117]:
chat_with_model(functions_list=functions_list)

模型回答: 你好！很高兴为你提供服务，有什么可以帮助你的吗？


您还有其他问题吗？(输入退出以结束对话):  请帮我查下邮箱里现在总共有多少封邮件


模型回答: 您的邮箱里共有14封邮件。


您还有其他问题吗？(输入退出以结束对话):  退出


至此，我们就完成了统计邮件总数的函数编写。当前外部函数编写的效率在extract_function_code函数加持下已经非常高了，我们只需要少量的人工来编写prompt、并围绕新函数进行测试，即可完成外部函数的编写。

&emsp;&emsp;但是，大家有没有想过，当前AI应用的开发流程其实还可以更加高效一些——面对大量潜在的未知用户需求（比如现在我想查看下邮箱里是否有陈明发来的未读邮件），我们或许可以借助大语言模型，即时将用户的需求翻译成外部函数创建的prompt，然后即时创建外部函数加入到Chat模型中来实时更新Chat模型能力，并最终提供更加完善的解决方案。这其实是一种更高级的自动化形式——大模型不仅可以编写功能实现的代码，而且需求到功能的过程也可以由大模型自己来进行总结。而若能做到这一点，此时这个产品的功能就相当于是可以实现自生长（根据用户的需求实时成长），毫无疑问，这样的一个开发过程，才更加贴近我们想象中的智能化开发过程。

> 就像《流量地球2》中能够实时生成操作系统的550W。

&emsp;&emsp;当然，需要说明的是，对于这样的一种大语言模型的高层次应用，截止目前，受限于大语言模型本身的能力，并不能完全应用于软件开发的各个方面（比如无法实时生成操作系统）。但是，尝试将大语言模型应用于创建功能自生长的AI应用当中，仍然是极具价值的尝试，哪怕在部分场景下能够实现需求到功能的自设计和函数自编写，都将为软件开流程带来革命性的效率提升，因此，我们也非常有必要在课程中介绍和讨论这部分大模型的应用方法。